In [1]:
import requests

In [2]:
from datetime import date, timedelta

In [3]:
#Cowin API request
cowin_headers = {
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36",
}

In [4]:
pincodes=[560076,400058,400051,400064]

In [5]:
available_centers = {
    'mumbai': {} ,
    'bangalore': {}
}

In [6]:
for i in range(0,3):
    reqd_date = date.today() + timedelta(days=i)
    search_date = reqd_date.strftime("%d-%m-%Y")
    available_centers['mumbai'][search_date], available_centers['bangalore'][search_date]=[], []
    for code in pincodes:
        response = requests.get(f"https://cdn-api.co-vin.in/api/v2/appointment/sessions/calendarByPin?pincode={code}&date={search_date}" ,headers = cowin_headers)
        if response.status_code==200:
            centers = response.json()['centers'] 
            results = list(filter(lambda x:  (len(list (filter( lambda y: ( y['available_capacity']>0 and y['min_age_limit']==18),x['sessions'] )  ))>0), centers))
            if len(results)>0:
                available_centers['mumbai'][search_date].append((code,[x['name'] for x in results])) \
                if int(str(code)[:1]) == 4 else \
                         available_centers['bangalore'][search_date].append((code,[x['name'] for x in results]))
                    
    if len(available_centers['mumbai'][search_date])==0:
        del available_centers['mumbai'][search_date]
    if len(available_centers['bangalore'][search_date])==0: 
        del available_centers['bangalore'][search_date]
    
                

In [7]:
#Message config
message_url = "https://www.fast2sms.com/dev/bulk"

my_data = {
     # Your default Sender ID
    'sender_id': 'FSTSMS', 
    
    'message': '', 
    
    'language': 'english',
    'route': 'p',

    # separated by comma.
    'numbers':  '',
}
  
message_headers = {
    'authorization': 'xDqak7et1LpB6jNsiKYo4TvCdw9r2mf0GAu5IzS3cOXQMyJWHnfkMvobSyPOKex3L2z1RN4HadV5thWT',
    'Content-Type': "application/x-www-form-urlencoded",
    'Cache-Control': "no-cache"
}

In [8]:
text_message={
    'mumbai':'',
    'bangalore':''
}

In [9]:
def print_message(slots,available_date,city):
    text_message[city] += '  Date:' + available_date[:5] + '  '
    for slot in slots:
        text_message[city] += 'Pincode: ' + str(slot[0])[-2:] + '  ' + ','.join(slot[1]) + '  '
        

In [10]:
for city in available_centers:
    for slot in available_centers[city]:
        [print_message(available_centers[city][slot], slot, city) for x in available_centers[city][slot]]

In [11]:
text_message

{'mumbai': '', 'bangalore': ''}

In [12]:
for city in text_message:
    if text_message[city]:
        my_data['message'] = text_message[city]
        my_data['numbers'] = {Comma separated string of numbers for mumbai} if city == 'mumbai' else {Comma separated string of numbers for Bangalore}
        requests.request("POST",message_url,data = my_data,headers = message_headers)